In [1]:
from os import path,getcwd
import pandas as pd
import numpy as np

from scipy import stats

import statsmodels.api as sm
from bioinfokit.analys import stat
from statsmodels.graphics.gofplots import qqplot
from sklearn.model_selection import train_test_split
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import pearsonr
from fitter import Fitter


from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"

- 1. Verificar se as amostras seguem uma Distribuição Normal, Se pertencem a mesma distribuição e se são Independentes. Plotar os gráficos.

- 2. DESAFIO: Inserir as estatísticas de Média e Desvio Padrão no gráfico


In [2]:
path.root = path.join(getcwd(),'..')
path.data = path.join(path.root,'data')
path.output = path.join(path.data,'output')
path.input = path.join(path.data,'input')

In [3]:
lista_estados = ['Chicago', 'HarrisburgScranton', 'Pittsburgh','Boise', 'LosAngeles', 'LasVegas', 'Atlanta','DallasFtWorth', 'MiamiFtLauderdale','StLouis','Syracuse', 'Sacramento', 'Boston', 'Charlotte','Spokane', 'Albany', 'Houston', 'SanFrancisco','RaleighGreensboro', 'BuffaloRochester','GrandRapids', 'Denver', 'SanDiego','BaltimoreWashington', 'Roanoke', 'RichmondNorfolk', 'Louisville', 'Seattle','CincinnatiDayton', 'NewYork', 'NewOrleansMobile', 'Indianapolis','Jacksonville', 'Columbus', 'Detroit', 'Philadelphia','PhoenixTucson', 'Nashville', 'Portland', 'HartfordSpringfield','Tampa', 'Orlando',]
lista_regioes = ['Southeast', 'Plains',  'Midsouth', 'NorthernNewEngland', 'Northeast', 'SouthCentral', 'GreatLakes', 'West','WestTexNewMexico',]

In [4]:
path_read = path.join(path.input,'avocado.csv')

names = ['date','average_price','total_volume','PLU_4046','PLU_4225','PLU_4770','total_bags','small_bags','large_bags','XLarge_bags','type','year','region']

dtypes = {
    'total_volume':np.int64,
    'PLU_4046':np.int64,
    'PLU_4225':np.int64,
    'PLU_4770':np.int64,
    'total_bags':np.int64,
    'small_bags':np.int64,
    'large_bags':np.int64,
    'XLarge_bags':np.int64
}


df = pd.read_csv(
        path_read,
        index_col=0,
        names=names,
        header=0,
        parse_dates=['date'],
)

df = df.astype(dtypes)

df = df.sort_values(
            by='date',
            ignore_index=True
)

df

,date,average_price,total_volume,PLU_4046,PLU_4225,PLU_4770,total_bags,small_bags,large_bags,XLarge_bags,type,year,region
0,2015-01-04,1.75,27365,9307,3844,615,13598,13061,537,0,organic,2015,Southeast
1,2015-01-04,1.49,17723,1189,15628,0,905,905,0,0,organic,2015,Chicago
2,2015-01-04,1.68,2896,161,206,0,2528,2528,0,0,organic,2015,HarrisburgScranton
3,2015-01-04,1.52,54956,3013,35456,1561,14925,11264,3660,0,conventional,2015,Pittsburgh
4,2015-01-04,1.64,1505,1,1129,0,374,186,187,0,organic,2015,Boise
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-03-25,1.36,908202,142681,463136,174975,127409,103579,22467,1362,conventional,2018,Chicago
18245,2018-03-25,0.70,9010588,3999735,966589,30130,4014132,3398569,546409,69152,conventional,2018,SouthCentral
18246,2018-03-25,1.42,163496,29253,5080,0,129163,109052,20111,0,organic,2018,SouthCentral
18247,2018-03-25,1.70,190257,29644,70982,0,89631,89424,207,0,organic,2018,California


In [5]:
mask = (df.type == 'conventional') & (df.region =='TotalUS')

df_average_price   =  df.loc[mask, 'average_price']

data_1 = df_average_price.sample(frac=0.3)
data_2 = df_average_price.sample(frac=0.3)


In [6]:
fig = ff.create_distplot(
    [data_1,data_2],
    group_labels=['amostra 1','amostra 2'],
    bin_size=.1,
    show_rug=True,
    colors = px.colors.qualitative.Pastel
)

fig.update_layout(
    height = 800
)

fig.show()

In [7]:
stat, p = ttest_ind(data_1, data_2)

print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Provavelmente a mesma distribuição')
else:
	print('Provavelmentes as distribuições são diferentes')
 

stat=0.060, p=0.952
Provavelmente a mesma distribuição


In [8]:
stat, p = shapiro(data_1)

print('stat=%.3f, p=%.3f' % (stat, p))

if p > 0.05:
	print('Provavelmente uma Distribuição Normal')
else:
	print('Provavelmente NÃO é uma Distribuição Normal')


stat=0.907, p=0.001
Provavelmente NÃO é uma Distribuição Normal


In [9]:
stat, p = pearsonr(data_1, data_2)

print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Provavelmente Independente')
else:
	print('Provavelmente Dependente')



stat=-0.280, p=0.046
Provavelmente Dependente


In [10]:
f = Fitter(data_1,)
f.fit()

fig = px.bar(f.y, labels=['amostra'])

counter = 0

list_colors = px.colors.qualitative.Pastel + px.colors.qualitative.Pastel2 + px.colors.qualitative.Set3 + px.colors.qualitative.Set2 + px.colors.qualitative.Set1 + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24  + px.colors.qualitative.Light24 + px.colors.qualitative.Dark24

name = list(f.get_best().keys())[0]

fig = px.bar(f.y, labels=['amostra'])

fig.add_trace(
    go.Scatter(
        y=f.fitted_pdf[name],
        name=name
    )
)
fig.show()


#for name in f.fitted_pdf:
#    fig.add_trace(
#        go.Scatter(
#            y=f.fitted_pdf[name],
#            name=name,
#            marker=dict(color = list_colors[counter])
#        )    
#    )
#    counter += 1
#fig.show()

C:\Python\python310_x64\lib\site-packages\scipy\integrate\quadpack.py:879: IntegrationWarning:

The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.

C:\Python\python310_x64\lib\site-packages\scipy\integrate\quadpack.py:879: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

